In [112]:
import pandas as pd
import numpy as np

In [113]:
df=pd.read_csv('../data/hotel_bookings.csv') #cargamos datos
print(df.columns) # veo columnas
df.head()

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [132]:
#df.notna().sum().sort_values(ascending=True)
 #compruebo integridad de los datos, 
 #elimino company, muy pocos datos, elimino filas con datos restandes de agente country y children

In [115]:
from sklearn.impute import SimpleImputer

imp=SimpleImputer(strategy='most_frequent')
df.agent=imp.fit_transform(np.array(df.agent).reshape(-1, 1))


df.drop(['company'],axis=1,inplace=True)
df.drop(['arrival_date_year'],axis=1,inplace=True)
df.drop(['reservation_status_date'],axis=1,inplace=True)
df.drop(['reservation_status'],axis=1,inplace=True)
df.drop(['assigned_room_type'],axis=1,inplace=True)
#df.drop(['country'],axis=1,inplace=True)
#df.drop(['lead_time'],axis=1,inplace=True)

df['children'].fillna(0,inplace=True)
df["meal"].replace({"Undefined": "SC", }, inplace=True) #las reservas con regimen sin definir las convertimos en reservas sin regimen 
df['country'].fillna('Unkown',inplace=True)
df['agent'].fillna(0,inplace=True)
filter = (df.children != 0) | (df.adults != 0) | (df.babies != 0)
df.children=df.children.astype(int)
df.adr=df.adr.astype(int)
df.agent=df.agent.astype(int)
df=df[filter]


df.is_canceled=df.is_canceled.astype('category')
df.is_repeated_guest=df.is_repeated_guest.astype('category')



In [131]:
#df.notna().sum().sort_values(ascending=True)
#comprobar que integridad de los datos

In [117]:
from sklearn.preprocessing import LabelEncoder
col_cat=df.select_dtypes(exclude=[np.number,'category']).columns.tolist()
print(col_cat)
le = LabelEncoder()
for i in col_cat:
    le.fit(df.loc[:,i])
    df.loc[:,i]=le.transform(df.loc[:,i])
    df.loc[:,i]=df.loc[:,i].astype('category')



['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'deposit_type', 'customer_type']


In [118]:
from sklearn.model_selection import train_test_split

df_train,df_test=train_test_split(df,test_size=0.1,random_state=42)
print(df_train.shape,df_test.shape)

(107289, 27) (11921, 27)


In [120]:
from sklearn.preprocessing import StandardScaler,RobustScaler,MaxAbsScaler

scaler=StandardScaler()
num_col=df.select_dtypes(include=np.number).columns.tolist()
print(num_col)
for i in num_col:
    scaler.fit(np.array(df_train.loc[:,i]).reshape(-1, 1))
    df_train.loc[:,i]=scaler.transform(np.array(df_train.loc[:,i]).reshape(-1, 1))
    df_test.loc[:,i]=scaler.transform(np.array(df_test.loc[:,i]).reshape(-1, 1))
#escalado de datos    

['lead_time', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests']


In [121]:
'''col_cat=df.select_dtypes(include=['category']).columns.tolist()
col_cat.remove('is_canceled')
df_train=pd.get_dummies(df_train,columns=col_cat,drop_first=True)
df_test=pd.get_dummies(df_test,columns=col_cat,drop_first=True)'''

#no cambia los resultados
#uso del onehot encoder no cambia los resultados y aumenta el tiempo de ejecucion, 
# necesario para xgboost

"col_cat=df.select_dtypes(include=['category']).columns.tolist()\ncol_cat.remove('is_canceled')\ndf_train=pd.get_dummies(df_train,columns=col_cat,drop_first=True)\ndf_test=pd.get_dummies(df_test,columns=col_cat,drop_first=True)"

In [122]:
'''col_cat=df.select_dtypes(include=['category']).columns.tolist()
le = LabelEncoder()
for i in col_cat:
    df_train.loc[:,i]=df_train.loc[:,i].astype('int')
    df_test.loc[:,i]=df_test.loc[:,i].astype('int')'''
    #en lugar de categorias usar enteros

"col_cat=df.select_dtypes(include=['category']).columns.tolist()\nle = LabelEncoder()\nfor i in col_cat:\n    df_train.loc[:,i]=df_train.loc[:,i].astype('int')\n    df_test.loc[:,i]=df_test.loc[:,i].astype('int')"

In [126]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
ada=AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=3))
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      7438
           1       0.84      0.76      0.80      4483

    accuracy                           0.86     11921
   macro avg       0.85      0.84      0.85     11921
weighted avg       0.86      0.86      0.86     11921



In [50]:
from xgboost import XGBClassifier
ada=XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

[16:53:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      7438
           1       0.86      0.77      0.81      4483

    accuracy                           0.87     11921
   macro avg       0.86      0.85      0.85     11921
weighted avg       0.87      0.87      0.86     11921



In [129]:
ada=RandomForestClassifier(max_features=1.0,n_estimators=1000)
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      7438
           1       0.88      0.83      0.85      4483

    accuracy                           0.89     11921
   macro avg       0.89      0.88      0.88     11921
weighted avg       0.89      0.89      0.89     11921



In [109]:
ada=GradientBoostingClassifier(learning_rate=1,max_depth=10,n_estimators=200)
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      7438
           1       0.85      0.81      0.83      4483

    accuracy                           0.87     11921
   macro avg       0.87      0.86      0.87     11921
weighted avg       0.87      0.87      0.87     11921



In [94]:
ada=KNeighborsClassifier(n_neighbors=1,weights='uniform')
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87      7438
           1       0.78      0.78      0.78      4483

    accuracy                           0.83     11921
   macro avg       0.82      0.82      0.82     11921
weighted avg       0.83      0.83      0.83     11921



In [80]:
ada=BaggingClassifier(max_features=1.0,n_estimators=200)
ada.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=ada.predict(df_test.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))
#igual que random forest

              precision    recall  f1-score   support

           0       0.90      0.93      0.92      7438
           1       0.88      0.83      0.85      4483

    accuracy                           0.89     11921
   macro avg       0.89      0.88      0.88     11921
weighted avg       0.89      0.89      0.89     11921



In [130]:
from sklearn.feature_selection import SelectKBest

aux=SelectKBest(k=15)
aux.fit_transform(df_train.drop(['is_canceled'],axis=1),df_train['is_canceled'])
k_filter = aux.get_support()
features = np.array(df_train.drop(['is_canceled'],axis=1).columns)
 

print("Selected best 15:")
print(features[k_filter])
(df_train[features[k_filter]].head())
df_train_15k=df_train[features[k_filter]]
df_train_15k['is_canceled']=df_train['is_canceled']
df_test_15k=df_test[features[k_filter]]
df_test_15k['is_canceled']=df_test['is_canceled']


Selected best 15:
['hotel' 'lead_time' 'adults' 'country' 'market_segment'
 'distribution_channel' 'is_repeated_guest' 'previous_cancellations'
 'previous_bookings_not_canceled' 'reserved_room_type' 'booking_changes'
 'deposit_type' 'customer_type' 'required_car_parking_spaces'
 'total_of_special_requests']


In [16]:
ada=AdaBoostClassifier(base_estimator = DecisionTreeClassifier())
ada.fit(df_train_15k.drop(['is_canceled'],axis=1),df_train_15k.is_canceled)
pred=ada.predict(df_test_15k.drop(['is_canceled'],axis=1))


print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      7438
           1       0.75      0.67      0.71      4483

    accuracy                           0.79     11921
   macro avg       0.78      0.77      0.77     11921
weighted avg       0.79      0.79      0.79     11921



In [17]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn import svm 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier



reg_log=Pipeline(steps= [ 
('reglog',LogisticRegression())
 ]   )

svc=Pipeline([ ('svc',svm.SVC())  ])

rand_forest=RandomForestClassifier()

adaB=AdaBoostClassifier()
knn=KNeighborsClassifier()
gb=GradientBoostingClassifier()
bag=BaggingClassifier()

re_log_param={
'reglog__penalty':['l1','l2'],
'reglog__C':np.arange(0.1,4,0.5)
}

svc_param={
'svc__C':np.arange(0.1,0.9,0.1),
'svc__kernel':['linear','poly','rbf']

}
rand_forest_param={
'n_estimators':[100,500,1000],
'max_features':[1.0,2,3]
 }

adaB_param={
'n_estimators':[10,100,500,1000],
'learning_rate':[0.01,0.1,0.25,0.5,1]
}

knn_param={
'n_neighbors':[1,3,5,7],
'weights':['uniform','distance']
}
gb_param={
'max_depth':[1,2,3],
'n_estimators':[100,200,500],
'learning_rate':[0.01,0.1,0.25,0.5,1]

}

bag_param={
'n_estimators':[50,100,200],
'max_samples':[10,50,100,200],
 'bootstrap':[True,False]
}


gs_reg_log=GridSearchCV(reg_log,re_log_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_svm=GridSearchCV(svc,svc_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_rand_forest=GridSearchCV(rand_forest,rand_forest_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_adaB=GridSearchCV(adaB,adaB_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_knn=GridSearchCV(knn,knn_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_gb=GridSearchCV(gb,gb_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)
gs_bag=GridSearchCV(bag,bag_param,cv=2,scoring='recall',n_jobs=-1,verbose=1)

grids={
'gs_reg_log':gs_reg_log,
#'gs_svm':gs_svm,
'gs_rand_forest':gs_rand_forest,
'gs_adaB':gs_adaB,
'gs_knn':gs_knn,
'gs_gb':gs_gb,
'gs_bag':gs_bag
}

In [217]:
#tarda más de una hora buscar todos los parametros!!!
for nombre,grid_search in grids.items(): 
    grid_search.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)

Fitting 2 folds for each of 16 candidates, totalling 32 fits
Fitting 2 folds for each of 18 candidates, totalling 36 fits
Fitting 2 folds for each of 20 candidates, totalling 40 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 45 candidates, totalling 90 fits
Fitting 2 folds for each of 24 candidates, totalling 48 fits


In [224]:
estimador=gs_rand_forest.best_estimator_
estimador.score(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=estimador.predict(df_test.drop(['is_canceled'],axis=1))
print(metrics.classification_report(df_test.is_canceled.values,pred))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90      7438
           1       0.88      0.77      0.82      4483

    accuracy                           0.87     11921
   macro avg       0.88      0.85      0.86     11921
weighted avg       0.88      0.87      0.87     11921



In [218]:
best_grids=[(i,j.best_score_) for i,j in grids.items()]
best_grids=pd.DataFrame(best_grids,columns=['Grid','Best Score'])
best_grids.sort_values(by='Best Score', ascending=False)

,Grid,Best Score
4,gs_gb,0.761028
3,gs_knn,0.754381
2,gs_adaB,0.656964
0,gs_reg_log,0.606456
5,gs_bag,0.570677
1,gs_rand_forest,0.022157


In [225]:
print(gs_rand_forest.best_params_)
print(gs_gb.best_params_)
gs_knn.best_params_
'''

{'max_features': 3, 'n_estimators': 1000}
{'learning_rate': 1, 'max_depth': 3, 'n_estimators': 200}
{'n_neighbors': 1, 'weights': 'uniform'}


'''

{'max_features': 3, 'n_estimators': 1000}


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [128]:
from sklearn.ensemble import VotingClassifier

rnd_clf=RandomForestClassifier(max_features=1.0,n_estimators=200)
gb_clf=GradientBoostingClassifier(learning_rate=1,max_depth=5,n_estimators=200)
knn_clf=KNeighborsClassifier(n_neighbors=1,weights='uniform')
ada=AdaBoostClassifier(base_estimator = DecisionTreeClassifier())


voting_clf= VotingClassifier(estimators=[('rnd',rnd_clf),('gb',gb_clf),('knn',knn_clf),('ada',ada)],
voting='soft')


voting_clf.fit(df_train.drop(['is_canceled'],axis=1),df_train.is_canceled)
pred=voting_clf.predict(df_test.drop(['is_canceled'],axis=1))
print(metrics.classification_report(df_test.is_canceled.values,pred))


              precision    recall  f1-score   support

           0       0.89      0.93      0.91      7438
           1       0.87      0.81      0.84      4483

    accuracy                           0.88     11921
   macro avg       0.88      0.87      0.87     11921
weighted avg       0.88      0.88      0.88     11921



In [19]:
metrics.confusion_matrix(df_test.is_canceled.values,pred)

array([[6914,  524],
       [ 994, 3489]], dtype=int64)